In [ ]:
import pandas as pd
import shap
import numpy as np

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler


In [ ]:
df = pd.read_csv('Cachexia_US_Dx.txt', sep=",")
df.head()


In [ ]:
df.dtypes

In [ ]:
df = df.drop(columns=['patid'])

In [ ]:
df

In [ ]:
df['group'].unique()

In [ ]:
df['group'] = df['group'].fillna(0)

In [ ]:
df['group'].unique()

In [ ]:
df['group'] = np.where(df['group']=='Dx', 1, df['group'])

In [ ]:
df['group'].unique()

In [ ]:
df = df[df["group"] != 'Pred'] 

In [ ]:
df['group'].unique()

In [ ]:
df = df.groupby('group').apply(lambda x: x.sample(500)).reset_index(drop=True)


In [ ]:
df['group'].unique()

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
#scaler = MinMaxScaler()

#df['age'] = scaler.fit_transform(df['age'].values.reshape(-1,1))
#df['Albumin'] = scaler.fit_transform(df['Albumin'].values.reshape(-1,1))
#df['Hemoglobin'] = scaler.fit_transform(df['Hemoglobin'].values.reshape(-1,1))
#df['CRP'] = scaler.fit_transform(df['CRP'].values.reshape(-1,1))


In [ ]:
df

In [ ]:
X = df.drop('group',axis=1)
Y = df.group


In [ ]:
X

In [ ]:
X = X.astype(float)
Y = Y.astype(float)

In [ ]:
Y

In [ ]:
X = X.values
Y = Y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_split=0.2)


In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  



In [ ]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
confusion

In [ ]:
display = ConfusionMatrixDisplay(confusion)
display.plot()

In [ ]:
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

In [ ]:
df

In [ ]:
shap.summary_plot(shap_values, X, feature_names=df.columns.drop(['group', 'F17', 'E66']), max_display=20)

In [ ]:
shap.plots.heatmap(shap_values, feature_values=shap_values.abs.max(0),  instance_order=shap_values.sum(1),  max_display=20)

In [ ]:
shap_values.data

In [ ]:
df

In [ ]:
import xgboost

In [ ]:
X = df.drop('group',axis=1)
Y = df.group


In [ ]:
model = xgboost.XGBClassifier(nestimators=341, max_depth=2).fit(X, Y)

In [ ]:
explainer = shap.Explainer(model, X)
shap_values = explainer(X[:1000])

In [ ]:
shap.plots.heatmap(shap_values, max_display=20, feature_values=shap_values.abs.max(0),  instance_order=shap_values.sum(1))
